In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libs
import logging
import pandas as pd
from collections import OrderedDict
import pickle
import torch

# Own Modules
#from source_torch.mlca.mlca import mlca_mechanism
#from source_torch.mlca.mlca_setup import set_value_model_parameters
from source_torch.mlca.mlca import mlca_mechanism
from source_torch.mlca.mlca_setup import set_value_model_parameters
import datetime
import os
from source_torch.util import save_result, load_result
import numpy as np

MLCA NN Class imported
MLCA NN_MIP Class imported
WDP Class imported
MLCA_Economies imported
MLCA function imported


In [3]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% |  0% |
|  5 |  0% |  0% |
|  6 | 97% | 86% |
|  7 |  0% |  0% |


In [4]:
torch.cuda.set_device(2)
torch.cuda.is_available()

True

In [5]:
iterations = 1
seed = 0

while iterations > 0:
    x = datetime.datetime.now()
    m = x.month
    d = x.day
    h = x.hour
    mi = x.minute

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    # log debug to console
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
#     logging.basicConfig(level=logging.WARNING, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
#     logging.basicConfig(level=logging.INFO, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')    
    
    configdict_mlca = OrderedDict([('SATS_domain_name','LSVM'),
                        ('SATS_auction_instance_seed', seed),
                        ('bidders',0),
                        ('items',0),
                        ('bidder_ids',0),
                        ('Qinit', 40),
                        ('Qmax', 50),
                        ('Qround',0),
                        ('Starter','mlca_extra'),
                        ('epochs' , 512),#512
                        ('batch_size' , 32),
                        ('regularization_type' , 'l1_l2'),
                        ('layer_N' , [10, 10, 10]),
                        ('layer_R' ,[32, 32] ),
                        ('layer_L' , []),
                        ('NN_parameters',[]),
                        ('bigM',6000),
                        ('warm_start',False),
                        ('MIP_parameters',[]),
                        ('scaler',None),
                        ('init_bids_and_fitted_scaler',[None,None]),
                        ('return_allocation',True),
                        ('return_payments',True),
                        ('lra',0.01),
                        ('regularization',1e-5),
                        ('calc_efficiency_per_iteration',True),
                        ('active_learning_algorithm', 'galo'),
                        ('presampled_n', 20),
                        ('presampled_algorithm', 'unif')])
    configdict_mlca = set_value_model_parameters(configdict_mlca)
    res = mlca_mechanism(configdict = configdict_mlca)
    seed += 1
    iterations -= 1
    if res == 'UNIQUENESS CHECK FAILED see logfile':
        iterations +=1
        continue
    dirname = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_'+ configdict_mlca['active_learning_algorithm']
    result_dir = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_' + configdict_mlca['active_learning_algorithm'] +'/' +str(m)+'_'+str(d) + '_' + str(h) + '_' + str(mi) + '_' + str(configdict_mlca['SATS_auction_instance_seed']-1)
    os.makedirs(dirname,exist_ok=True)
    save_result(result_dir, res)

20:20:07:               START MLCA:
20:20:07:               -----------------------------------------------
20:20:07:               Model: LSVM
20:20:07:               Seed SATS Instance: 2
20:20:07:               Qinit: 40
20:20:07:               Qmax: 50
20:20:07:               Qround: 6

20:20:07:               Instantiate SATS Instance



------------------------ SATS parameters ------------------------
Value Model: LSVM
Number of Bidders:  6
Number of BidderTypes:  2
Number of Items:  18
Scaler:  None

------------------------ DNN  parameters ------------------------
Epochs: 512
Batch Size: 32
Regularization: l1_l2

Bidder_0:
regularization: 1e-05
learning_rate: 0.01
architecture: [10, 10, 10]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_1:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_2:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_3:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_4:
regularization: 1e-05
learning_rate: 0.01
archite

20:20:08:               Set NN parameters
20:20:08:               Set MIP parameters
20:20:08:               INITIALIZE BIDS
20:20:08:               -----------------------------------------------

20:20:08:               Set up initial Bids with GALI Learning Method for: Bidder_0
20:20:08:               Sampling with GALI 40 bundle-value pairs from bidder 0
20:21:01:               Set up initial Bids with GALI Learning Method for: Bidder_1
20:21:01:               Sampling with GALI 40 bundle-value pairs from bidder 1
20:21:54:               Set up initial Bids with GALI Learning Method for: Bidder_2
20:21:54:               Sampling with GALI 40 bundle-value pairs from bidder 2
20:22:47:               Set up initial Bids with GALI Learning Method for: Bidder_3
20:22:47:               Sampling with GALI 40 bundle-value pairs from bidder 3
20:23:36:               Set up initial Bids with GALI Learning Method for: Bidder_4
20:23:36:               Sampling with GALI 40 bundle-value pairs f

First time in optimization step.


20:25:47:               Tighten bounds with IA for Bidder_0
20:25:47:               Tighten bounds with IA for Bidder_2
20:25:47:               Tighten bounds with IA for Bidder_3
20:25:47:               Tighten bounds with IA for Bidder_4
20:25:47:               Tighten bounds with IA for Bidder_5
20:25:47:               Sorted active bidders in MIP: ['Bidder_0', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5']
20:25:48:               Mip initialized
20:25:48:               Solving MIP
20:25:48:               Attempt no: 1
20:25:48:               Mip time Limit of 1800
20:25:48:               Mip relative gap 0.001
20:25:48:               Mip integrality tol 1e-08


...

01:11:04:               Tighten bounds with IA for Bidder_0
01:11:04:               Tighten bounds with IA for Bidder_1
01:11:04:               Tighten bounds with IA for Bidder_2
01:11:04:               Tighten bounds with IA for Bidder_3
01:11:04:               Tighten bounds with IA for Bidder_4
01:11:04:               Tighten bounds with IA for Bidder_5
01:11:04:               Sorted active bidders in MIP: ['Bidder_0', 'Bidder_1', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5']
01:11:04:               Adding bidder specific constraints
01:11:04:               Mip initialized
01:11:04:               Solving MIP
01:11:04:               Attempt no: 1
01:11:04:               Mip time Limit of 1800
01:11:04:               Mip relative gap 0.001
01:11:04:               Mip integrality tol 1e-08


Time limit is 1800


01:11:23:               -----------------------------------------------
01:11:23:               Status  : integer optimal, tolerance
01:11:23:               Time    : 19 sec
01:11:23:               Problem : MILP
01:11:23:               Rel. Gap: 0.00099 %
01:11:23:               N. Iter : 653599
01:11:23:               Hit Lim.: False
01:11:23:               SET ARGMAX ALLOCATION ONLY BIDDER SPECIFIC for Bidder_4
01:11:23:               Bidder_0:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] | [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
01:11:23:               Bidder_1:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] | [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
01:11:23:               Bidder_2:  [0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0.] | [0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
01:11:23:               Bidder_3:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] | [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.

Time for MIP run: 19 sec
Found objective 632.0589800852717


01:11:23:               Status  : integer optimal solution
01:11:23:               Time    : 0 sec
01:11:23:               Problem : MILP
01:11:23:               Rel. Gap: 0.0 %
01:11:24:               N. Iter : 24
01:11:24:               Hit Lim.: False
01:11:24:               Objective Value: 587.0604950257846
01:11:24:               ALLOCATION
01:11:24:               ---------------------------------------------
01:11:24:               Calculate MLCA allocation: Main Economy
01:11:24:               Active bidders: ['Bidder_0', 'Bidder_1', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5']
01:11:24:               MLCA allocation in Main Economy:
01:11:24:               Bidder_0 {'good_ids': [], 'value': 0}
01:11:24:               Bidder_1 {'good_ids': [], 'value': 0}
01:11:24:               Bidder_2 {'good_ids': [1, 6, 7, 8, 12, 13, 14, 15], 'value': 297.06143327752164}
01:11:24:               Bidder_3 {'good_ids': [2], 'value': 8.185066054338987}
01:11:24:               Bidder_4 {'good

In [1]:
#import numpy as np
#np.mean(scores), np.std(scores)
stats

NameError: name 'stats' is not defined

In [8]:
save_result(dirname, stats)

'Result is saved as./experiments/MLCA/Torch/results6_20_12_56'

In [25]:
dirname_active_lsvm = './experiments/MLCA/Torch/results'+'19'+'_'+'6' + '_' + '17' + '_' + '7'
dirname_active_mrvm = './experiments/MLCA/Torch/results'+'6'+'_'+'20' + '_' + '12' + '_' + '56'
dirname_active_gsvm = './experiments/MLCA/Torch/results'+'6'+'_'+'20' + '_' + '18' + '_' + '25'
dirname_unif_lsvm = './experiments/MLCA/Torch/results'+'6'+'_'+'20' + '_' + '19' + '_' + '51'
loaded_res_1 = load_result(dirname_active_lsvm)
loaded_res_2 = load_result(dirname_active_mrvm)
loaded_res_3 = load_result(dirname_active_gsvm)
loaded_res_4 = load_result(dirname_unif_lsvm)
results_all = [loaded_res_1, loaded_res_2, loaded_res_3, loaded_res_4]

./experiments/MLCA/Torch/results19_6_17_7 is loaded 
./experiments/MLCA/Torch/results6_20_12_56 is loaded 
./experiments/MLCA/Torch/results6_20_18_25 is loaded 
./experiments/MLCA/Torch/results6_20_19_51 is loaded 


In [31]:
loaded_res_1[0]
#loaded_res[0]['MLCA Allocation']
# for res_l in loaded_res:
#     try:
#         print(res_l['MLCA Efficiency'])
#     except:
#         pass
# loaded_res[0]['MLCA Efficiency']

{'SEED TRUE VALUES': {'Seed': 0,
  'SCW': 569.8218784366154,
  'Allocation': {0: {'good_ids': [], 'value': 0.0},
   1: {'good_ids': [], 'value': 0.0},
   2: {'good_ids': [2, 3, 4, 9, 14, 15], 'value': 163.48956471299064},
   3: {'good_ids': [], 'value': 0.0},
   4: {'good_ids': [0, 1, 6, 7, 8, 12, 13], 'value': 247.01856363364843},
   5: {'good_ids': [5, 10, 11, 16, 17], 'value': 159.31375008997634}}},
 'MLCA Efficiency': 0.8845542910694237,
 'MLCA Allocation': OrderedDict([('Bidder_0', {'good_ids': [], 'value': 0}),
              ('Bidder_1', {'good_ids': [], 'value': 0}),
              ('Bidder_2', {'good_ids': [], 'value': 0}),
              ('Bidder_3',
               {'good_ids': [2, 3, 4, 8, 9, 17], 'value': 146.89721960144118}),
              ('Bidder_4',
               {'good_ids': [0, 1, 6, 7, 12, 13, 14],
                'value': 197.40095824774474}),
              ('Bidder_5',
               {'good_ids': [5, 10, 11, 15, 16],
                'value': 159.74020986716178})]),
 

In [32]:
loaded_res[0]['Statistics']['Efficiency per Iteration']

OrderedDict([(0, 0.6880173599333909), (1, 0.8845542910694237)])

In [20]:
loaded_res[0]['Statistics']['Efficient allocation per Iteration']

OrderedDict([(0,
              OrderedDict([('Bidder_0',
                            {'good_ids': [0,
                              1,
                              2,
                              3,
                              4,
                              6,
                              7,
                              8,
                              10,
                              11,
                              12,
                              13,
                              14,
                              15,
                              16,
                              17],
                             'value': 392.0473444342457}),
                           ('Bidder_1', {'good_ids': [], 'value': 0}),
                           ('Bidder_2', {'good_ids': [], 'value': 0}),
                           ('Bidder_3', {'good_ids': [], 'value': 0}),
                           ('Bidder_4', {'good_ids': [], 'value': 0}),
                           ('Bidder_5', {'good_ids'

In [27]:
#LSVM
#MRVM
#GSVM
for i_load in results_all:
    mean_scores = []
    for res_l in i_load:
        try:
            mean_scores.append(res_l['MLCA Efficiency'])
            #print(res_l['MLCA Efficiency'])
        except:
            pass
    print(np.mean(mean_scores), np.std(mean_scores))

0.8398288549098961 0.05182647459019574
0.8031062808481103 0.040352068583338244
0.9337057142031697 0.04263073235248043
0.7507823537018439 0.07330770972253216
